In [35]:
# Data Handle
import pandas as pd
import numpy as np
import random

# Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sns.set_style("whitegrid")

# Machine Learning - Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder

# Machine Learning - Metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as mape

# Machine Learning - Models
from sklearn.ensemble import RandomForestRegressor
#!pip install lightgbm
from lightgbm import LGBMRegressor as LGBM

# Machine Learning - Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Miscellaneous
import os
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

Cargamos los datos de logs_entrenamiento y notas_por_subseccion

In [36]:
logs = pd.read_csv('logs_entrenamiento.csv')
notas = pd.read_csv('otas_por_subseccion.csv')
logs_val = pd.read_csv('logs_validacion.csv')

In [37]:
notas

,Unnamed: 0,Username,Grade,Grade Scaled,Quiz 1: Uso de modelos matemáticos en la economía,Quiz 2: Motivación y Función de Producción,Quiz 3: Preferencias y Toma de Decisiones,Quiz 4: Salario y Efectos,Quiz 5: Explicando Diferencias y Conclusión,Quiz 6: Introducción y conceptos iniciales,...,Quiz 21: Impuestos y Subsidios,Quiz 22: Eficiencia y externalidades,Quiz 23: Corrección de fallas,Quiz 24: Otras fuentes de ineficiencia,Quiz 25: Regla de la mayoría,Quiz 26: La paradoja de Condorcet,Quiz 27: Teorema del votante mediano,Quiz 28: Decisiones intertemporales,Quiz 29: Tópicos Adicionales,Quiz (Avg)
0,3,05743498d061bf472df897f1e5bdda6d62e99312c952e3...,0.94,6.6,0.75,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.905172
1,4,2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...,1.00,7.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.965517
2,5,c5f32ddce577c3f4cce3bcdcb5c8e0e03933b5cc62fa18...,0.99,6.9,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.956897
3,6,81528304670b5be911641ecc2ba3a21195234d10c3ab43...,1.00,7.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.965517
4,7,4427f64ffb5a05abefc1aa6936fca72e2f8fc24ad5f11d...,0.91,6.3,Not Attempted,1.0,1.0,1.0,1.0,1.0,...,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,0.439655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,605,65a4a3bd05c0e27b7bba2ee5b95f50c9af51ba46bcdd17...,0.59,4.0,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.568966
583,607,aaaf669774ef0c97d31b015d5f49dabf34b7fc43f1c57e...,0.63,4.2,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,1.0,...,1.0,1.0,1.0,1.0,0.5,0.0,0.0,0.5,0.0,0.603448
584,608,1e1188e0478a823fc3f411ef6baed0f6abc324d5b70e60...,0.51,3.6,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,1.0,...,1.0,Not Attempted,Not Attempted,Not Attempted,1.0,1.0,Not Attempted,Not Attempted,Not Attempted,0.454138
585,610,d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...,0.00,1.0,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,...,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,Not Attempted,0.000000


In [38]:
notas.columns

Index(['Unnamed: 0', 'Username', 'Grade', 'Grade Scaled',
       'Quiz 1: Uso de modelos matemáticos en la economía',
       'Quiz 2: Motivación y Función de Producción',
       'Quiz 3: Preferencias y Toma de Decisiones',
       'Quiz 4: Salario y Efectos',
       'Quiz 5: Explicando Diferencias y Conclusión',
       'Quiz 6: Introducción y conceptos iniciales',
       'Quiz 7: Prediciendo el resultado de un juego',
       'Quiz 8: Evaluación de Resultados',
       'Quiz 9: Determinación de Asignaciones', 'Quiz 10: Desigualdad',
       'Quiz 11: Conceptos centrales', 'Quiz 12: Profundizando lo aprendido',
       'Quiz 13: Conceptos centrales', 'Quiz 14: Profundizando lo aprendido',
       'Quiz 15: Conceptos centrales', 'Quiz 16: Profundizando lo aprendido',
       'Quiz 17: Introducción',
       'Quiz 18: Oferta, demanda y equilibrio de mercado',
       'Quiz 19: Cambios en la oferta y la demanda',
       'Quiz 20: Excedentes y la mano invisible',
       'Quiz 21: Impuestos y Subsidi

En primer lugar vamos a probar convirtiendo la columna event_type a dummies para después sumar los valores binarios por username.

In [39]:
# Utiliza la función get_dummies para convertir "event_type" en dummies
dummies = pd.get_dummies(logs['event_type'], prefix='event')
dummies_val = pd.get_dummies(logs_val['event_type'], prefix='event')

In [40]:
# Combina las dummies con tu DataFrame original
df = pd.concat([logs, dummies], axis=1)
df_val = pd.concat([logs_val, dummies_val], axis=1)

# Elimina la columna original "event_type" si es necesario
df.drop('event_type', axis=1, inplace=True)
df_val.drop('event_type', axis=1, inplace=True)

In [41]:
# Ordenar por "time" de más antiguo a más reciente y luego por "username"
df = df.sort_values(by=['time', 'username'])

In [42]:
# Extraer usernames y labels
usernames_labels = df[['username', 'label']].drop_duplicates()
#usernames_labels_val = df_val[['username', 'label']].drop_duplicates()

# Supongamos que tienes todas las columnas binarias en tu DataFrame df
# y deseas excluir algunas columnas de la suma
columnas_a_excluir = ['time', 'grouped_event_type', 'chapter', 'sequential']  # Lista de columnas a excluir

# Genera la lista de todas las columnas binarias
columnas_binarias = df.columns[df.columns.str.startswith('event_')]

# Luego, elimina las columnas que deseas excluir de la suma
columnas_a_sumar = [col for col in columnas_binarias if col not in columnas_a_excluir] 

# Ahora puedes usar columnas_a_sumar en el groupby
df_grouped = df.groupby('username')[columnas_a_sumar].sum().reset_index()
#df_grouped_val = df_val.groupby('username')[columnas_a_sumar].sum().reset_index()


In [43]:
# Combina los DataFrames por "username"
df_grouped = df_grouped.merge(usernames_labels, on='username', how='inner')
#df_grouped_val = df_grouped_val.merge(usernames_labels_val, on='username', how='inner')

In [44]:
# Lista de columnas que comienzan con "Quiz" (excluyendo 'Quiz (Avg)')
columnas_quiz = [col for col in notas.columns if col.startswith('Quiz') and col != 'Quiz (Avg)']

# Reemplaza 'Not Attempted' por -1 en las columnas de Quiz
notas[columnas_quiz] = notas[columnas_quiz].replace('Not Attempted', -1)

In [45]:
# Combina los DataFrames por "username"
df_entrenamiento = df_grouped.merge(notas, left_on='username', right_on='Username', how='inner')
#df_validacion = df_grouped_val.merge(notas, left_on='username', right_on='Username', how='inner')

In [46]:
df_entrenamiento.to_csv('df_entrenamiento.csv', index=False)
#df_validacion.to_csv('df_validacion', index=False)